In [60]:
import scala.util.matching
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.{min, max, mean, stddev, typedLit, when}
import org.apache.spark.sql.types.{IntegerType, DoubleType}
//import spark.implicits._ 

println("all good")

all good


data is in the format:

{
"conversionSymbol": "",
"volumeto": 6685400.72, 
"high": 9165.89, 
"low": 9144.6, 
"time": 1595030400, 
"volumefrom": 730.39, 
"close": 9151.68, 
"open": 9156.79, 
"conversionType": "direct"
}


In [3]:
case class TradingInfo (time: Int,
                        high: Double,
                        low: Double,
                        open: Double,
                        volumefrom: Double,
                        volumeto: Double, 
                        close: Double,
                        conversionType: String,
                        conversionSymbol: String                        
                       )

def createTradingInfo(data: String): TradingInfo = {
    val s = data.split(", ")
    return TradingInfo(s(0).split(": ")(1).toInt, s(1).split(": ")(1).toDouble, s(2).split(": ")(1).toDouble, 
                       s(3).split(": ")(1).toDouble, s(4).split(": ")(1).toDouble, s(5).split(": ")(1).toDouble, 
                       s(6).split(": ")(1).toDouble, s(7).split(": ")(1), s(8).split(": ")(1))
}

val filename = "../Datasets/bitcoin_json.txt"
//val johnFileName = "//home/john/ID2221-Labs/PreProcessing/"
val tradingInformation = sc.textFile(filename).
            flatMap(l => l.split("},")).
            map(x => x.replace("[", "")).
            map(x => x.replace("{", "")).
            map(x => x.replace("\"", "")).
            map(createTradingInfo)
            .cache()

tradingInformation.take(3).foreach(println)

println("All good")

TradingInfo(1595466000,9540.53,9515.47,9524.38,1555.82,1.483120278E7,9538.04,\direct\,\\)
TradingInfo(1595469600,9545.07,9512.26,9538.04,1358.04,1.293527787E7,9514.73,\direct\,\\)
TradingInfo(1595473200,9514.84,9486.07,9514.73,1372.42,1.303854406E7,9508.16,\direct\,\\)
All good


defined class TradingInfo
filename = ../Datasets/bitcoin_json.txt
tradingInformation = MapPartitionsRDD[13] at map at <console>:46


createTradingInfo: (data: String)TradingInfo


MapPartitionsRDD[13] at map at <console>:46

In [119]:
// Data Action


// val df = tradingInformation.toDF("conversionSymbol","volumeto", "high", "low", "time", 
//                                  "volumefrom", "close","open","conversionType").show(5)


import org.apache.spark.sql.functions.when

val df = spark.createDataFrame(tradingInformation).sort("time")
val df1 = df.withColumn("midPrice",($"high" + $"low")/2)

// df1.show(5)

val ratio: Double = 0.7
val training_size = df_1.count*ratio
val testing_size = df_1.count - 10

val training_position = df_1.take(training_size.toInt).last.getInt(0)
val testing_position = df_1.take(testing_size.toInt).last.getInt(0)

val df2 = df1.withColumn("dataset", when(col("time") <= training_position,"training")
      .when(col("time") > training_position and col("time") <= testing_position,"validation")
      .otherwise("testing"))


// val training_set = df_1.filter($"time" <= training_position).cache()
// val validation_set = df_1.filter($"time" > training_position).filter($"time" <= testing_position).cache()
// val testing_set = df_1.filter($"time" > testing_position).cache()


// training_set.show(5)
// validation_set.show(5)
// testing_set.show(5)

df2.show(5)

println("All good")

+----------+-------+------+-------+----------+--------+-------+--------------+----------------+-----------------+--------+
|      time|   high|   low|   open|volumefrom|volumeto|  close|conversionType|conversionSymbol|         midPrice| dataset|
+----------+-------+------+-------+----------+--------+-------+--------------+----------------+-----------------+--------+
|1388581200|  967.0|718.46|  812.9|     1.012|  889.99|  967.0|      \direct\|              \\|           842.73|training|
|1388584800| 759.11|750.13| 750.13|     1.661| 1282.36| 759.11|      \direct\|              \\|           754.62|training|
|1388588400|1169.16|912.66|1169.16|    0.5916|  575.07|1000.26|      \direct\|              \\|          1040.91|training|
|1388592000| 759.11|745.01| 759.11|    0.9084|  841.07| 745.01|      \direct\|              \\|           752.06|training|
|1388595600| 750.13|745.01| 745.01|    0.4054|  340.39| 750.13|      \direct\|              \\|747.5699999999999|training|
+----------+----

df = [time: int, high: double ... 7 more fields]
df1 = [time: int, high: double ... 8 more fields]
ratio = 0.7
training_size = 41884.5
testing_size = 59825
training_position = 1539360000
testing_position = 1602626400
df2 = [time: int, high: double ... 9 more fields]


[time: int, high: double ... 9 more fields]

In [117]:
val statistics = df_1.groupBy("dataset").agg(mean("midPrice"))

// , stddev("midPrice"), min("midPrice"), max("midPrice"),
//                          mean("volumefrom"), stddev("volumefrom"), min("volumefrom"), max("volumefrom"),
//                          mean("volumeto"), stddev("volumeto"), min("volumeto"), max("volumeto")

// val midPrice_mean = statistics.getDouble(0)
// val midPrice_std = statistics.getDouble(1)
// val midPrice_min = statistics.getDouble(2)
// val midPrice_max = statistics.getDouble(3)
// val midPrice_range = midPrice_max - midPrice_min

// // // VolumeFrom
// val volumefrom_mean = statistics.getDouble(4)
// val volumefrom_std = statistics.getDouble(5)
// val volumefrom_min = statistics.getDouble(6)
// val volumefrom_max = statistics.getDouble(7)
// val volumefrom_range = volumefrom_max - volumefrom_min

// // // VolumeTo
// val volumeto_mean = statistics.getDouble(8)
// val volumeto_std = statistics.getDouble(9)
// val volumeto_min = statistics.getDouble(10)
// val volumeto_max = statistics.getDouble(11)
// val volumeto_range = volumeto_max - volumeto_min

// // from -1 to 1 
// //val df_2 = df_1.withColumn("midPriceNorm",($"midPrice" - midPrice_mean)/midPrice_std)
// // from 0 to 1 
// val df_2 = df_1.withColumn("midPriceNorm",(($"midPrice" - midPrice_min)/midPrice_range).cast(DoubleType))
//                .withColumn("volumefromNorm",(($"volumefrom" - volumefrom_min)/volumefrom_range).cast(DoubleType))
//                .withColumn("volumetoNorm",(($"volumeto" - volumeto_min)/volumeto_range).cast(DoubleType))
//                .cache()

// df_2.show(5)
// // df_2.printSchema()

println("All good")

lastException = null


org.apache.spark.sql.AnalysisException: Cannot resolve column name "dataset" among (time, high, low, open, volumefrom, volumeto, close, conversionType, conversionSymbol, midPrice);

In [6]:
// Saving data as Text

// don't forget to delete the previous folder 
// RDD does not allow us to overwrite: https://community.cloudera.com/t5/Support-Questions/Apache-SPARK-Overwrite-data-file/m-p/105253
val df_3 = df_2.withColumn("empty", typedLit(Seq(1)))
df_3.rdd.repartition(1).saveAsTextFile("data_cleaned")

println("All good")

Unknown Error: <console>:37: error: not found: value df_2
       val df_3 = df_2.withColumn("empty", typedLit(Seq(1)))
                  ^
